# NLP
### Text 데이터를 분석하고 모델링하는 분야 : 자연어처리(NLP)
#### 자연어를 이해하는 영역(NLU) + 모델이 자연어를 생성하는 영역(NLG) = NLP

#### 다양한 Task
1. 감정분석
2. 요약
3. 기계 번역
4. 질문 응답

In [1]:
S1 = '나는 책상 위에 사과를 먹었다'       
S2 = '알고 보니 그 사과는 Jason 것이었다' 
S3 = '그래서 Jason에게 사과를 했다'

S4 = '나는 책상 위에 배를 먹었다'

## Tokenization

In [2]:
print(S1.split())
print(S2.split())
print(S3.split()) 

['나는', '책상', '위에', '사과를', '먹었다']
['알고', '보니', '그', '사과는', 'Jason', '것이었다']
['그래서', 'Jason에게', '사과를', '했다']


In [3]:
print(list(S1))

['나', '는', ' ', '책', '상', ' ', '위', '에', ' ', '사', '과', '를', ' ', '먹', '었', '다']


In [4]:
token2idx = {}
index = 0

for sentence in [S1, S2, S3]:
    tokens = sentence.split()
    for token in tokens:
        if token2idx.get(token) == None:
            token2idx[token] = index
            index += 1

print(token2idx)

{'나는': 0, '책상': 1, '위에': 2, '사과를': 3, '먹었다': 4, '알고': 5, '보니': 6, '그': 7, '사과는': 8, 'Jason': 9, '것이었다': 10, '그래서': 11, 'Jason에게': 12, '했다': 13}


In [5]:
def indexed_sentence(sentence):
    return [token2idx[token] for token in sentence]
    
S1_i = indexed_sentence(S1.split())
print(S1_i)

S2_i = indexed_sentence(S2.split())
print(S2_i)

S3_i = indexed_sentence(S3.split())
print(S3_i)

[0, 1, 2, 3, 4]
[5, 6, 7, 8, 9, 10]
[11, 12, 3, 13]


## OOV(out-of-vocabulary

In [6]:
S4 = '나는 책상 위에 배를 먹었다'

indexed_sentence(S4.split())
# KeyError: '배를'

KeyError: '배를'

In [7]:
S4 = '나는 책상 위에 배를 먹었다'

# 기존 token 사전에 <unk> token 추가
token2idx = {t : i+1 for t, i in token2idx.items()}
token2idx['<unk>'] = 0

# token이 없을 경우, <unk> token의 0을 치환
def indexed_sentence_unk(sentence):
    return [token2idx.get(token, token2idx['<unk>']) for token in sentence]

indexed_sentence_unk(S4.split())

[1, 2, 3, 0, 5]

문장을 띄어쓰기 단위가 아닌 글자로 token사용 => OOV현상 방지  
그러나 글자 하나에는 의미가 거의 없음.  
짧은 문장이나 단어를 이용하는 task가 아니면 글자 기반 tokenizer는 어려움

## n-gram
여러 개(n)의 연속된 윈도우를 단위로 살펴보는 방법  
n>=4이면 n-gram

In [8]:
S1 = '나는 책상 위에 사과를 먹었다'

print([S1[i:i+1] for i in range(len(S1))]) # uni-gram(n=1)
print([S1[i:i+2] for i in range(len(S1))]) # bi -gram(n=2)
print([S1[i:i+3] for i in range(len(S1))]) # tri-gramb(b=3)

['나', '는', ' ', '책', '상', ' ', '위', '에', ' ', '사', '과', '를', ' ', '먹', '었', '다']
['나는', '는 ', ' 책', '책상', '상 ', ' 위', '위에', '에 ', ' 사', '사과', '과를', '를 ', ' 먹', '먹었', '었다', '다']
['나는 ', '는 책', ' 책상', '책상 ', '상 위', ' 위에', '위에 ', '에 사', ' 사과', '사과를', '과를 ', '를 먹', ' 먹었', '먹었다', '었다', '다']


In [9]:
S5 = 'I am dying to play the game'
S5_sp = S5.split()

print([" ".join(S5_sp[i:i+1]) for i in range(len(S5_sp))]) # uni-gram
print([" ".join(S5_sp[i:i+2]) for i in range(len(S5_sp))]) # bi -gram
print([" ".join(S5_sp[i:i+3]) for i in range(len(S5_sp))]) # tri-gram

['I', 'am', 'dying', 'to', 'play', 'the', 'game']
['I am', 'am dying', 'dying to', 'to play', 'play the', 'the game', 'game']
['I am dying', 'am dying to', 'dying to play', 'to play the', 'play the game', 'the game', 'game']


그러나 무의미한 조합들이 너무 많이 생김.  

n-gram의 이점과 의미있는 것들만 token으로 사용하는 방법 => BPE

## BPE
자주 나오는 글자의 나열에 치환을 이용하여 효율적인 token사용  
1. 띄어쓰기 기반 tokenization
2. 연속된 2개의 글자의 숫자를 세어 가장 많이 나오는 글자 2개의조합 찾기
3. 두 글자를 합쳐 기존 사전의 단어 수정
4. 미리 정해 놓은 횟수만큼 2~3번의 과정 반복

In [9]:
#Algorithm 1: Learn BPE operations
import re, collections
def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i], symbols[i+1]] += freq
            '''Step 1
            {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, 
            ('w', 'e'): 8, ('e', 'r'): 2, ('r', '</w>'): 2, 
            ('n', 'e'): 6, ('e', 'w'): 6, ('e', 's'): 9, 
            ('s', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, 
            ('i', 'd'): 3, ('d', 'e'): 3}'''
    return pairs
def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\\S)'+bigram+r'(?!\\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

vocab = {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w e s t </w>': 6, 'w i d e s t </w>':3} #1
num_merges = 10
for i in range(num_merges): #4
    pairs = get_stats(vocab) #2
    best = max(pairs, key=pairs.get) #2
    vocab = merge_vocab(best, vocab) #3
    print(f'Step {i+1}')
    print(best)
    print(vocab)
    print('')

Step 1
('e', 's')
{'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w es t </w>': 6, 'w i d es t </w>': 3}

Step 2
('es', 't')
{'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est </w>': 6, 'w i d est </w>': 3}

Step 3
('est', '</w>')
{'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est</w>': 6, 'w i d est</w>': 3}

Step 4
('l', 'o')
{'lo w </w>': 5, 'lo w e r </w>': 2, 'n e w est</w>': 6, 'w i d est</w>': 3}

Step 5
('lo', 'w')
{'low </w>': 5, 'low e r </w>': 2, 'n e w est</w>': 6, 'w i d est</w>': 3}

Step 6
('n', 'e')
{'low </w>': 5, 'low e r </w>': 2, 'ne w est</w>': 6, 'w i d est</w>': 3}

Step 7
('ne', 'w')
{'low </w>': 5, 'low e r </w>': 2, 'new est</w>': 6, 'w i d est</w>': 3}

Step 8
('new', 'est</w>')
{'low </w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}

Step 9
('low', '</w>')
{'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}

Step 10
('w', 'i')
{'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'wi d est</w>': 3}



In [4]:
S1 = '나는 책상 위에 사과를 먹었다'       
S2 = '알고 보니 그 사과는 Jason 것이었다' 
S3 = '그래서 Jason에게 사과를 했다'

token_counts = {}
index = 0

for sentence in [S1, S2, S3]:
    tokens = sentence.split()
    for token in tokens:
        if token_counts.get(token) == None:
            token_counts[token] = 1
        else:
            token_counts[token] += 1

token_counts = {" ".join(token) : counts for token, counts in token_counts.items()}
print(token_counts)

{'나 는': 1, '책 상': 1, '위 에': 1, '사 과 를': 2, '먹 었 다': 1, '알 고': 1, '보 니': 1, '그': 1, '사 과 는': 1, 'J a s o n': 1, '것 이 었 다': 1, '그 래 서': 1, 'J a s o n 에 게': 1, '했 다': 1}


In [10]:
num_merges = 10

for i in range(num_merges):
    pairs = get_stats(token_counts)
    best = max(pairs, key=pairs.get)
    token_counts = merge_vocab(best, token_counts)
    print(f'Step {i + 1}')
    print(best)
    print(token_counts)
    print('\n')

Step 1
('사', '과')
{'나 는': 1, '책 상': 1, '위 에': 1, '사과 를': 2, '먹 었 다': 1, '알 고': 1, '보 니': 1, '그': 1, '사과 는': 1, 'J a s o n': 1, '것 이 었 다': 1, '그 래 서': 1, 'J a s o n 에 게': 1, '했 다': 1}


Step 2
('사과', '를')
{'나 는': 1, '책 상': 1, '위 에': 1, '사과를': 2, '먹 었 다': 1, '알 고': 1, '보 니': 1, '그': 1, '사과 는': 1, 'J a s o n': 1, '것 이 었 다': 1, '그 래 서': 1, 'J a s o n 에 게': 1, '했 다': 1}


Step 3
('었', '다')
{'나 는': 1, '책 상': 1, '위 에': 1, '사과를': 2, '먹 었다': 1, '알 고': 1, '보 니': 1, '그': 1, '사과 는': 1, 'J a s o n': 1, '것 이 었다': 1, '그 래 서': 1, 'J a s o n 에 게': 1, '했 다': 1}


Step 4
('J', 'a')
{'나 는': 1, '책 상': 1, '위 에': 1, '사과를': 2, '먹 었다': 1, '알 고': 1, '보 니': 1, '그': 1, '사과 는': 1, 'Ja s o n': 1, '것 이 었다': 1, '그 래 서': 1, 'Ja s o n 에 게': 1, '했 다': 1}


Step 5
('Ja', 's')
{'나 는': 1, '책 상': 1, '위 에': 1, '사과를': 2, '먹 었다': 1, '알 고': 1, '보 니': 1, '그': 1, '사과 는': 1, 'Jas o n': 1, '것 이 었다': 1, '그 래 서': 1, 'Jas o n 에 게': 1, '했 다': 1}


Step 6
('Jas', 'o')
{'나 는': 1, '책 상': 1, '위 에': 1, '사과를': 2, '먹 었다': 1, '알 고': 1, '보 니': 

## 5-2. pre-trained tokenizer 사용하기

In [1]:
# 5-5_model_imdb_BERT.ipynb Code 확인
!pip install transformers
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

print(len(tokenizer.vocab))

     ---------------------------------------- 6.3/6.3 MB 2.3 MB/s eta 0:00:00
     -------------------------------------- 190.3/190.3 kB 2.3 MB/s eta 0:00:00
     -------------------------------------- 268.0/268.0 kB 4.2 MB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 2.8 MB/s eta 0:00:00
     -------------------------------------- 153.2/153.2 kB 1.8 MB/s eta 0:00:00


C:\Users\leeej\anaconda3\envs\practice\lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\leeej\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


30522


In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
sentence = "My dog is cute. He likes playing"
print(tokenizer.tokenize(sentence))
#split과 다르지 않은 결과

['my', 'dog', 'is', 'cute', '.', 'he', 'likes', 'playing']


In [13]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
print(len(tokenizer.vocab))
print(tokenizer.tokenize(sentence))
#학습한 데이터에 따라 tokenizer가 달라짐.
#'##e'는 앞에 띄어쓰기가 아닌 바로 이어지는 token의미

105879
['my', 'dog', 'is', 'cut', '##e', '.', 'he', 'likes', 'playing']


In [15]:
sentence = '나는 책상 위에 사과를 먹었다. 알고 보니 그 사과는 Jason 것이었다. 그래서 Jason에게 사과를 했다'       
print(tokenizer.tokenize(sentence))
#multilingual model에는 한국어 데이터도 포함

['나는', 'ᄎ', '##ᅢᆨ', '##상', '위에', 'ᄉ', '##ᅡ', '##과', '##를', 'ᄆ', '##ᅥ', '##ᆨ', '##었다', '.', '알', '##고', 'ᄇ', '##ᅩ', '##니', '그', 'ᄉ', '##ᅡ', '##과', '##는', 'jason', '것이', '##었다', '.', '그', '##래', '##서', 'jason', '##에게', 'ᄉ', '##ᅡ', '##과', '##를', '했다']


## Word-Embedding

## One-hot encoding

In [16]:
S1 = '나는 책상 위에 사과를 먹었다'        
S2 = '알고 보니 그 사과는 Jason 것이었다'  
S3 = '그래서 Jason에게 사과를 했다'       

token2idx = {}
index = 0

for sentence in [S1, S2, S3]:
    tokens = sentence.split()
    for token in tokens:
        if token2idx.get(token) == None:
            token2idx[token] = index
            index += 1

print(token2idx)

{'나는': 0, '책상': 1, '위에': 2, '사과를': 3, '먹었다': 4, '알고': 5, '보니': 6, '그': 7, '사과는': 8, 'Jason': 9, '것이었다': 10, '그래서': 11, 'Jason에게': 12, '했다': 13}


In [17]:
#python list를 이용해 모든 token을 원-핫 인코딩으로 표현하는 방법
V = len(token2idx) #14

token2vec = [([0 if i != idx else 1 for i in range(V)], idx, token) for token, idx in token2idx.items() ]

for x in token2vec:
    print("\t".join([str(y) for y in x]))

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]	0	나는
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]	1	책상
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]	2	위에
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]	3	사과를
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]	4	먹었다
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]	5	알고
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]	6	보니
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]	7	그
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]	8	사과는
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]	9	Jason
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]	10	것이었다
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]	11	그래서
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]	12	Jason에게
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]	13	했다


In [24]:
#python numpy를 이용해 문장을 원-핫 인코딩으로 바꾸는 방법
import numpy as np

for sentence in [S1, S2, S3]:
    onehot_s = []
    tokens = sentence.split()
    for token in tokens:
        if token2idx.get(token) != None:
            vector = np.zeros((1,V))
            vector[:,token2idx[token]] = 1
            onehot_s.append(vector)
        else:
            print("UNK")

    print(f"{sentence} : ")        
    print(np.concatenate(onehot_s, axis = 0))
    print('\n')

나는 책상 위에 사과를 먹었다 : 
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


알고 보니 그 사과는 Jason 것이었다 : 
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]


그래서 Jason에게 사과를 했다 : 
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


